In [ ]:
!mkdir -p historical_data

!curl -L https://gist.githubusercontent.com/nicholascc/39df3671b93665861a69e295bcaa50ca/raw/93cd0147900fc822b758ea6df0d19ac4c41b8c95/orders.csv -o historical_data/orders.csv
!curl -L https://gist.githubusercontent.com/nicholascc/39df3671b93665861a69e295bcaa50ca/raw/93cd0147900fc822b758ea6df0d19ac4c41b8c95/trades.csv -o historical_data/trades.csv

!echo "Files downloaded successfully to historical_data/"

In [ ]:
import pandas as pd

# Load data
orders = pd.read_csv('historical_data/orders.csv', parse_dates=['timestamp'])
trades = pd.read_csv('historical_data/trades.csv', parse_dates=['transaction_timestamp'])

# Process each market
market_grids = {}
for market in orders['market_name'].unique():
    # Filter market data
    mkt_orders = orders[orders['market_name'] == market].set_index('timestamp')
    mkt_trades = trades[trades['market_name'] == market].set_index('transaction_timestamp')
    
    # Create minute grids
    order_grid = mkt_orders.groupby(pd.Grouper(freq='1Min')).last()
    trade_grid = mkt_trades.groupby(pd.Grouper(freq='1Min')).agg({
        'price': 'last',
        'size': 'last'
    }).rename(columns={'price': 'last_trade_price', 'size': 'last_trade_size'})
    
    # Combine and store
    market_grids[market] = pd.concat([order_grid, trade_grid], axis=1).fillna(method='ffill')

# Example: Look at first market's data
first_market = list(market_grids.keys())[0]
print(f"\nSample of {first_market} market grid:")
display(market_grids[first_market].head())